In [1]:
import logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)
import torch
import torch.nn.functional as F

In [2]:
from german_parser.util.const import CONSTS
from german_parser.util.dataloader import TigerDatasetGenerator
import dill as pickle
from string import punctuation

In [3]:
# character_flag_generators = [
#             lambda c: int(c.isupper()),
#             lambda c: int(c.lower() in ["ä", "ö", "ü", "ß"]),
#             lambda c: int(c.isdigit()),
#             lambda c: int(c in punctuation)
#         ]
# dataset_generator = TigerDatasetGenerator(f"{CONSTS['data_dir']}/tiger/tiger_2.2_utf8.xml", (0.2, 0.1), character_flag_generators=character_flag_generators) # type:ignore

# required_vars = (*dataset_generator.get_training_dataloader(batch_size=32, shuffle=True), dataset_generator.character_set, dataset_generator.character_flag_generators, dataset_generator.inverse_word_dict)

# pickle.dump(required_vars, open("required_vars.pkl", "wb"))

In [4]:
train_dataloader, train_new_words, character_set, character_flag_generators, inverse_word_dict = pickle.load(open("required_vars.pkl", "rb"))

In [5]:
from german_parser.model import TigerModel

In [7]:
model = TigerModel(TigerModel.WordEmbeddingParams(char_set=character_set, char_flag_generators=character_flag_generators, char_internal_embedding_dim=10, char_part_embedding_dim=100, word_part_embedding_dim=100, char_internal_window_size=3, word_dict=inverse_word_dict), TigerModel.LSTMParams(hidden_size=512, bidirectional=True), TigerModel.LSTMParams(hidden_size=512, bidirectional=False))
model.cuda()
sum([p.numel() for p in model.parameters()])

16205163

In [7]:
optim = torch.optim.Adam(model.parameters(), lr=1e-2)

In [8]:
i = 0

while True:
    i += 1
    model.train()
    optim.zero_grad()
    input = next(iter(train_dataloader))

    words, sentence_lengths, target_heads = input
    
    words = words.cuda()
    target_heads = target_heads.cuda()

    self_attention, labels, indices = model((words, sentence_lengths), train_new_words)
    loss = F.cross_entropy(self_attention[indices], target_heads[indices])

    print(f"iteration {i} loss {loss.item():.6f}")

    loss.backward()

    optim.step()
    

iteration 1 loss 3.170976
iteration 2 loss 3.009099
iteration 3 loss 3.284932
iteration 4 loss 2.960798
iteration 5 loss 3.036205
iteration 6 loss 3.179747
iteration 7 loss 3.141706
iteration 8 loss 3.155389
iteration 9 loss 2.963340
iteration 10 loss 3.074751


KeyboardInterrupt: 